# Getting started with TensorFlow

**目的**
  - Tensor の定義と基本的な演算の仕方を学ぶ
  - `tf.function`の使い方とメリットを学ぶ
  - Tensorflow の自動微分機能について学ぶ
  - スクラッチで定義した線形回帰モデルのトレーニングの仕方を学ぶ

このノートブックでは、TensorFlowでのTensorの演算と、TensoFlowでの変数の操作について学びます。また、Pythonのリスト型やnumpy arrayとの互換性についても確認します。

その後で、簡単な線形回帰のモデルを、コアTensorFlowを使って（= Keras等のハイレベルAPIを使わずに）スクラッチで実装します。


## 必要なパッケージのインストールとバージョンの確認

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
import tensorflow as tf

In [ ]:
print(tf.__version__)

## Tensor の操作

### 変数(Variables) と定数(Constants)

TensorFlow上では、Tensorは定数 (`tf.constant`) か変数 (`tf.Variable`) のどちらかで定義されます。
定数は一度定義すると変更できませんが、変数は値を自由に変更することができます。

そのため、`tf.Variable`は値を変更するための様々なメソッドを持っていますが、`tf.constant`は持っていません。

`tf.Variable` で定義された `x` を変更するためには、以下のようなメソッドを利用します。

* `x.assign(new_value)`
* `x.assign_add(value_to_be_added)`
* `x.assign_sub(value_to_be_subtracted`



In [ ]:
x = tf.constant([2, 3, 4])
x

In [ ]:
x = tf.Variable(2.0, dtype=tf.float32, name='my_variable')

In [ ]:
x.assign(45.8)
x

In [ ]:
x.assign_add(4).assign_sub(3)
x

### 成分ごとの演算

Tensorflow では、numpy のように成分毎に演算を適応することができます。

* `tf.add`: 加算 
* `tf.multiply`: 乗算
* `tf.subtract`: 減算 
* `tf.math.*`:  その他の様々な演算

また、通常の演算子(`+`, `-`, etc.)を用いることでも、同様の結果を得ることができます。


In [ ]:
a = tf.constant([5, 3, 8])
b = tf.constant([3, -1, 2])
c = tf.add(a, b)
d = a + b

print("c:", c)
print("d:", d)

In [ ]:
a = tf.constant([5, 3, 8])
b = tf.constant([3, -1, 2])
c = tf.multiply(a, b)
d = a * b

print("c:", c)
print("d:", d)

In [ ]:
# tf.math.exp expects floats so we need to explicitly give the type
a = tf.constant([5, 3, 8], dtype=tf.float32)
b = tf.math.exp(a)

print("b:", b)

### Python 型、NumPy との互換性

TensorFlow のネイティブの Tensor 以外にも、Python の型や numpy array を演算に用いることもできます。


In [ ]:
# native python list
a_py = [1, 2] 
b_py = [3, 4] 

In [ ]:
tf.add(a_py, b_py)

In [ ]:
# numpy arrays
a_np = np.array([1, 2])
b_np = np.array([3, 4])

In [ ]:
tf.add(a_np, b_np)

In [ ]:
# native TF tensor
a_tf = tf.constant([1, 2])
b_tf = tf.constant([3, 4])

In [ ]:
tf.add(a_tf, b_tf)

また、 `.numpy()` を利用して、TF Tensor を numpy array に変換することも可能です。

In [ ]:
a_tf.numpy()

## 線形回帰

それでは、TensorFlow の低レベルAPIを使って、線形回帰のモデルを　実装してみましょう

より抽象度の高い高レベルAPI (Keras) については、このコースの後半で学習します。


### トイデータセット

では、以下の関数をモデル化しましょう。

\begin{equation}
y= 2x + 10
\end{equation}

In [ ]:
X = tf.constant(range(10), dtype=tf.float32)
Y = 2 * X + 10

print("X:{}".format(X))
print("Y:{}".format(Y))

モデルを検証するためのテストデータセットも作成しましょう。

In [ ]:
X_test = tf.constant(range(10, 20), dtype=tf.float32)
Y_test = 2 * X_test + 10

print("X_test:{}".format(X_test))
print("Y_test:{}".format(Y_test))

#### 損失関数 (Loss Function)

$X$ から $Y$ を予測するための単純なモデルとして、$Y$の平均を返すだけのモデルを定義してみます。

In [ ]:
y_mean = Y.numpy().mean()

def predict_mean(X):
    y_hat = [y_mean] * len(X)
    return y_hat

Y_hat = predict_mean(X_test)

二乗誤差平均 (MSE, Mean Square Error)を使って、損失 (Loss) を計算してみましょう。

\begin{equation}
MSE = \frac{1}{m}\sum_{i=1}^{m}(\hat{Y}_i-Y_i)^2
\end{equation}

この単純なモデルの損失は、以下のように計算できます。

In [ ]:
errors = (Y_hat - Y)**2
loss = tf.reduce_mean(errors)
loss.numpy()

この MSE の値は、他のモデルのパフォーマンスと比較する際のベースラインとしておきましょう。

$\hat{Y}$ を線形回帰モデルの予測とすると、

\begin{equation}
\hat{Y} = w_0X + w_1
\end{equation}

引数にモデルの係数を取り、損失関数を以下のように書くこともできます。


In [ ]:
def loss_mse(X, Y, w0, w1):
    Y_hat = w0 * X + w1
    errors = (Y_hat - Y)**2
    return tf.reduce_mean(errors)

### 勾配の計算

勾配降下法を使用するためには、それぞれの重み毎 ($w_0$ と $w_1$) に損失関数の偏微分を計算する必要があります。

もちろん手動で行うこともできますが、Tensorflow には自動微分の機能が備わっているため、自分でコードを書く必要はありません！<br>
この機能は、勾配の情報を保存する`tf.GradientTape`インスタンスの中で損失の計算を包み込むことで利用することができます。

```python
with tf.GradientTape() as tape:
    loss = # 損失の計算 
```

このようにすることで、 `tf.GradientTape` の中で計算されたすべてのTensorの勾配を後のステップでとして計算することができます。

```python
gradients = tape.gradient(loss, [w0, w1])
```

In [ ]:
def compute_gradients(X, Y, w0, w1):
    with tf.GradientTape() as tape:
        loss = loss_mse(X, Y, w0, w1)
    return tape.gradient(loss, [w0, w1])

In [ ]:
w0 = tf.Variable(0.0)
w1 = tf.Variable(0.0)

dw0, dw1 = compute_gradients(X, Y, w0, w1)

In [ ]:
print("dw0:", dw0.numpy())

In [ ]:
print("dw1", dw1.numpy())

### トレーニングループ

ここでは、ごくシンプルなトレーニングループを定義しましょう。ミニバッチ、テストセットの分割、重みのランダムな初期化などのベストプラクティスは単純化のために一旦無視します。

In [ ]:
STEPS = 1000
LEARNING_RATE = .02
MSG = "STEP {step} - loss: {loss}, w0: {w0}, w1: {w1}\n"


w0 = tf.Variable(0.0)
w1 = tf.Variable(0.0)


for step in range(0, STEPS + 1):

    dw0, dw1 = compute_gradients(X, Y, w0, w1)
    w0.assign_sub(dw0 * LEARNING_RATE)
    w1.assign_sub(dw1 * LEARNING_RATE)

    if step % 100 == 0:
        loss = loss_mse(X, Y, w0, w1)
        print(MSG.format(step=step, loss=loss, w0=w0.numpy(), w1=w1.numpy()))


では、この線形回帰モデルの損失と、常に平均値を返すだけのベースラインモデルの損失と比較してみましょう

In [ ]:
loss = loss_mse(X_test, Y_test, w0, w1)
loss.numpy()

以前のモデルよりずっとよくなりました。

Copyright 2021 Google Inc. Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0 Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License